In [197]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import numpy as np
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    
    from elasticsearch import helpers
    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

Loaded  .. . . . . . . .


In [198]:
df = pd.read_excel("elastic_Search_Data.xlsx")
print("read")

read


In [199]:
df.head()

,filename,country,Product name,Refined Product Name,para_text,Possible Questions,Comment
0,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is indicated f...,What is ATACAND used for\nWhat is ATACAND used...,NaN
1,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Geriatrics (> 65 years of age) : No overall di...,What is the efficacy of ATACAND in elderly pop...,NaN
2,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Pediatrics (6 to 17 years of age): \n•\tHypert...,What is ATACAND used for in children\nWhat is ...,NaN
3,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is contraindic...,Who is ATACAND conracindicated in\nWho is ATAC...,NaN
4,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),There is evidence that co-administration of an...,Can ATACAND be co-administered with aliskiren\...,NaN


In [200]:
tfidf = TfidfVectorizer()

In [201]:
answers=list(df["para_text"])

In [202]:
answers

['ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'Geriatrics (> 65 years of age) : No overall differences in safety or effectiveness were observed between these subjects and younger subjects, and other reported clinical experience has not identified differences in responses between the elderly and younger patients, but greater sensitivity of some older individuals cannot be ruled out. ',
 'Pediatrics (6 to 17 years of age): \n•\tHypertension\nATACAND is indicated for the treatment of essential hypertension in children and adolescents 6 to 17 years of age (se

In [203]:
ml_vec= tfidf.fit_transform(answers).toarray()

In [204]:
tfidf.get_feature_names()

['000',
 '10',
 '1000',
 '12',
 '13',
 '14c',
 '15',
 '16',
 '17',
 '178',
 '18',
 '19',
 '213',
 '23',
 '235',
 '24',
 '240',
 '26',
 '2c9',
 '32',
 '3200',
 '33',
 '36',
 '37',
 '3x',
 '40',
 '50',
 '54x',
 '59',
 '60',
 '626',
 '65',
 '67',
 '672',
 '677',
 '73m2',
 '80',
 '8694',
 '8700',
 '90',
 '99',
 'ability',
 'abnormal',
 'about',
 'above',
 'absolute',
 'absorption',
 'accident',
 'according',
 'account',
 'accumulate',
 'ace',
 'aceis',
 'achieved',
 'across',
 'act',
 'action',
 'active',
 'activity',
 'acute',
 'added',
 'addition',
 'additional',
 'adequately',
 'adjusted',
 'adjustment',
 'administered',
 'administration',
 'adolescent',
 'adolescents',
 'adrenal',
 'adult',
 'adults',
 'adverse',
 'adversely',
 'ae',
 'aes',
 'affect',
 'affecting',
 'affinity',
 'after',
 'afterload',
 'age',
 'aged',
 'agents',
 'agranulocytosis',
 'albuminuria',
 'aldosterone',
 'aliskiren',
 'all',
 'allergy',
 'alone',
 'also',
 'alt',
 'alternative',
 'an',
 'anaesthesia',
 'and'

In [205]:
len(ml_vec[0])

1025

In [206]:
ml_vec[4]

array([0., 0., 0., ..., 0., 0., 0.])

In [207]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "ml_vector":{
         "type":"dense_vector",
         "dims":1025
      } 
    }
   }
}

In [208]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

python client created


In [209]:
#creating the database
IndexName = 'tfidingss'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [210]:
my

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [tfidingss/EwS7Z1lSSUehgaaVexKV7Q] already exists',
    'index_uuid': 'EwS7Z1lSSUehgaaVexKV7Q',
    'index': 'tfidingss'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [tfidingss/EwS7Z1lSSUehgaaVexKV7Q] already exists',
  'index_uuid': 'EwS7Z1lSSUehgaaVexKV7Q',
  'index': 'tfidingss'},
 'status': 400}

In [211]:
df.columns

Index(['filename', 'country', 'Product name', 'Refined Product Name',
       'para_text', 'Possible Questions', 'Comment'],
      dtype='object')

In [212]:
df22 = df.to_dict('records')
df22

[{'filename': 'Candesartan_PM_Reviewed.docx',
  'country': 'Canada',
  'Product name': 'Candesartan cilexetil',
  'Refined Product Name': 'Atacand (Candesartan Cilexetil)',
  'para_text': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
  'Possible Questions': 'What is ATACAND used for\nWhat is ATACAND used for in Canada\nHow can ATACAND be used\nHow can ATACAND be used in Canada\nCan ATACAND be used with calcium channel blockers\nCan ATACAND be used with calcium channel blockers in Canada\nCan ATACAND (candesartan cilexetil) be indicated for heartfailure',
  

In [213]:
len(df22)

99

In [214]:
arr=[]
for c, line in enumerate(df22):
    arr.append({"answer":line.get("para_text",""),
    'ml_vector':ml_vec[c]})

In [215]:
arr[0]

{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'ml_vector': array([0., 0., 0., ..., 0., 0., 0.])}

In [216]:
arr[0]

{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'ml_vector': array([0., 0., 0., ..., 0., 0., 0.])}

In [217]:
for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [218]:
resp

{'_index': 'tfidingss',
 '_type': '_doc',
 '_id': '98',
 '_version': 4,
 'result': 'updated',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 395,
 '_primary_term': 1}

In [219]:
#testing
que = "what is ATACAND?"
que_vec= tfidf.transform([que]).toarray()

In [220]:
que_vec

array([[0., 0., 0., ..., 0., 0., 0.]])

In [221]:
len(que_vec[0])

1025

In [222]:

script_query = {
"script_score": {
"query": {"match_all":{}},
"script": {
"source": "cosineSimilarity(params.query_vector, doc['ml_vector']) ",
"params": {"query_vector":que_vec[0]}
}}}

In [223]:
es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})

/home/ayush/anaconda3/envs/biobertnew/lib/python3.6/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{'took': 666,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 99, 'relation': 'eq'},
  'max_score': 0.23225324,
  'hits': [{'_index': 'tfidingss',
    '_type': '_doc',
    '_id': '66',
    '_score': 0.23225324,
    '_source': {'answer': 'The recommended initial dose of ATACAND is 16 mg, once daily when used as monotherapy. Total daily doses of ATACAND should range from 8- 32 mg. Doses >32 mg do not appear to have a greater effect on BP reduction, and there is relatively little experience with such doses. Most of the antihypertensive effect is present within 2 weeks and the maximal BP reduction is generally obtained within 4 weeks. For patients with possible depletion of intravascular volume (e.g. patients treated with diuretics, particularly those with impaired renal function) consideration should be given to administration of a lower dose. If BP is not controlled by ATACAND alone, a thiazide diuretic may be added 